In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik
    
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
#importing Misc Libraries
import os
import gc
import pickle
from datetime import datetime

import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import datetime

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# 0. Prepare functions

In [2]:
def load_data (path='../dataset/'):
    global application_train, application_test, POS_CASH_balance
    application_train = pd.read_csv(path + 'application_train.csv')
    application_test = pd.read_csv(path + 'application_test.csv')
    POS_CASH_balance = pd.read_csv(path + 'POS_CASH_balance.csv')
    print('Done loading')

# 1. Load Data

In [3]:
load_data()

Done loading


In [4]:
# Merging POS_CASH_balance with TARGET
print("-"*100)
print("Merging TARGET with POS_CASH_balance Table")
pos_cash_merged = application_train[['SK_ID_CURR', 'TARGET']].merge(POS_CASH_balance, on='SK_ID_CURR', how='left')
print("-"*100)

----------------------------------------------------------------------------------------------------
Merging TARGET with POS_CASH_balance Table
----------------------------------------------------------------------------------------------------


# 2. Data Cleaning & FE

In [5]:

def cleaning_pos_cash(df):
    # Sorting the dataframe according to the month of status from oldest to latest
    df = df.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=False)
    
    # Replace invalid values with NaN
    values_to_replace = ['XNA', 'XAP']
    df.replace(values_to_replace, np.nan, inplace=True)
    
    # Group NAME_CONTRACT_STATUS into 3 categories: 'Active', 'Completed', and 'Others'
    df['NAME_CONTRACT_STATUS'] = df['NAME_CONTRACT_STATUS'].apply(lambda status: 
        'Active' if status in ['Active', 'Demand'] else 
        ('Completed' if status in ['Completed', 'Canceled'] else 'Others')
    )
    
    # Handle NaN values with the fill_missing function
    df = fill_missing(df)
    
    # Step 1: Handle CNT_INSTALMENT_FUTURE == 0 and NAME_CONTRACT_STATUS == 'Active'
    df.loc[(df['CNT_INSTALMENT_FUTURE'] == 0) & (df['NAME_CONTRACT_STATUS'] == 'Active'), 'NAME_CONTRACT_STATUS'] = 'Completed'
    
    # Step 2: Handle CNT_INSTALMENT < CNT_INSTALMENT_FUTURE
    df = df[df['CNT_INSTALMENT'] >= df['CNT_INSTALMENT_FUTURE']]  # Drop rows where this condition holds
    
    # Step 3: Handle SK_DPD > 2920
    df.loc[df['SK_DPD'] > 2920, 'SK_DPD'] = np.nan  # or df.drop(df[df['SK_DPD'] > 2920].index)
    
    # Step 4: Handle SK_DPD_DEF > 2920
    df.loc[df['SK_DPD_DEF'] > 2920, 'SK_DPD_DEF'] = np.nan  # or df.drop(df[df['SK_DPD_DEF'] > 2920].index)
    
    # Step 5: Handle 'Completed' contracts with SK_DPD_DEF > 0
    df = df[~((df['NAME_CONTRACT_STATUS'] == 'Completed') & (df['SK_DPD_DEF'] > 0))]
    
    # Step 6: Handle 'Completed' contracts with CNT_INSTALMENT_FUTURE > 0
    df = df[~((df['NAME_CONTRACT_STATUS'] == 'Completed') & (df['CNT_INSTALMENT_FUTURE'] > 0))]
    
    # Step 7: Handle illogical relationships between SK_DPD and SK_DPD_DEF
    df.loc[df['SK_DPD_DEF'] > df['SK_DPD'], 'SK_DPD_DEF'] = np.nan  # Drop or set SK_DPD_DEF to NaN if illogical
    
    return df

def fill_missing(df):
    # Impute missing values by group (e.g., by 'MONTHS_BALANCE')
    df['MONTH_BALANCE_BINS'] = pd.cut(df['MONTHS_BALANCE'], 10)
    
    # Columns to impute
    col_to_impute = ['CNT_INSTALMENT_FUTURE', 'CNT_INSTALMENT']
    
    for col in col_to_impute:
        grouped = df.groupby('MONTH_BALANCE_BINS')[col]
        for val_to_group, group_data in grouped:
            if df[col].dtypes in [int, float]:
                mean_value = group_data.mean()
                df.loc[(df['MONTH_BALANCE_BINS'] == val_to_group) & (df[col].isna()), col] = mean_value
            else:
                mode_value = group_data.mode().iloc[0] if not group_data.mode().empty else np.nan
                df.loc[(df['MONTH_BALANCE_BINS'] == val_to_group) & (df[col].isna()), col] = mode_value
    
    df.drop(columns='MONTH_BALANCE_BINS', inplace=True)
    
    return df


In [6]:
def feature_engineering_pos_cash(df):
    
    # Create new binary features based on SK_DPD (late payment status)
    df['LATE_PAYMENT'] = df['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    df['POS_IS_DPD'] = df['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
    df['POS_IS_DPD_UNDER_120'] = df['SK_DPD'].apply(lambda x: 1 if (x > 0) & (x < 120) else 0)
    df['POS_IS_DPD_OVER_120'] = df['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)
    
    # Computing Exponential Moving Average (EMA) for some features based on MONTHS_BALANCE
    columns_for_ema = ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']
    exp_columns = ['EXP_' + ele for ele in columns_for_ema]
    df[exp_columns] = df.groupby('SK_ID_PREV')[columns_for_ema].transform(lambda x: x.ewm(alpha=0.6).mean())
    
    # Creating new features based on domain knowledge
    df['SK_DPD_RATIO'] = df['SK_DPD'] / (df['SK_DPD_DEF'] + 0.00001)
    df['TOTAL_TERM'] = df['CNT_INSTALMENT'] + df['CNT_INSTALMENT_FUTURE']
    df['EXP_POS_TOTAL_TERM'] = df['EXP_CNT_INSTALMENT'] + df['EXP_CNT_INSTALMENT_FUTURE']
    
    return df

In [7]:
#Aggregations over SK_ID_PREV
def aggregations_sk_id_prev(pos_cash):
    # Aggregating over SK_ID_PREV
    overall_aggregations = {
        'SK_ID_CURR': ['first'],
        'MONTHS_BALANCE': ['max'],
        'CNT_INSTALMENT': ['mean', 'max', 'min'],
        'CNT_INSTALMENT_FUTURE': ['mean', 'max', 'min'],
        'SK_DPD': ['max', 'sum'],
        'SK_DPD_DEF': ['max', 'sum'],
        'EXP_CNT_INSTALMENT': ['last'],
        'EXP_CNT_INSTALMENT_FUTURE': ['last'],
        'SK_DPD_RATIO': ['mean', 'max'], 
    }
    
    aggregations_for_year = {
        'CNT_INSTALMENT': ['mean', 'max', 'min'],
        'CNT_INSTALMENT_FUTURE': ['mean', 'max', 'min'],
        'SK_DPD': ['max', 'sum'],
        'SK_DPD_DEF': ['max', 'sum'],
        'EXP_CNT_INSTALMENT': ['last'],
        'EXP_CNT_INSTALMENT_FUTURE': ['last'],
        'SK_DPD_RATIO': ['mean', 'max'],
    }
    
    aggregations_for_categories = {
        'CNT_INSTALMENT': ['mean', 'max', 'min'],
        'CNT_INSTALMENT_FUTURE': ['mean', 'max', 'min'],
        'SK_DPD': ['max', 'sum'],
        'SK_DPD_DEF': ['max', 'sum'],
        'EXP_CNT_INSTALMENT': ['last'],
        'EXP_CNT_INSTALMENT_FUTURE': ['last'],
        'SK_DPD_RATIO': ['mean', 'max'],
    }

    # Performing overall aggregations over SK_ID_PREV
    pos_cash_aggregated_overall = pos_cash.groupby('SK_ID_PREV').agg(overall_aggregations)
    pos_cash_aggregated_overall.columns = ['_'.join(ele).upper() for ele in pos_cash_aggregated_overall.columns]
    pos_cash_aggregated_overall.rename(columns={'SK_ID_CURR_FIRST': 'SK_ID_CURR'}, inplace=True)

    # Yearwise aggregations
    pos_cash['YEAR_BALANCE'] = pos_cash['MONTHS_BALANCE'] // 12
    pos_cash_aggregated_year = pd.DataFrame()
    for year in range(2):
        group = pos_cash[pos_cash['YEAR_BALANCE'] == year].groupby('SK_ID_PREV').agg(aggregations_for_year)
        group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in group.columns]
        if year == 0:
            pos_cash_aggregated_year = group
        else:
            pos_cash_aggregated_year = pos_cash_aggregated_year.merge(group, on='SK_ID_PREV', how='outer')

    # Aggregating over SK_ID_PREV for rest of the years
    pos_cash_aggregated_rest_years = pos_cash[pos_cash['YEAR_BALANCE'] >= 2].groupby('SK_ID_PREV').agg(aggregations_for_year)
    pos_cash_aggregated_rest_years.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in pos_cash_aggregated_rest_years.columns]
    
    # Merging all the year-wise aggregations
    pos_cash_aggregated_year = pos_cash_aggregated_year.merge(pos_cash_aggregated_rest_years, on='SK_ID_PREV', how='outer')
    pos_cash = pos_cash.drop(['YEAR_BALANCE'], axis=1)

    # Aggregating over SK_ID_PREV for each of NAME_CONTRACT_STATUS categories
    contract_type_categories = ['Active', 'Completed', 'Others']
    pos_cash_aggregated_contract = pd.DataFrame()
    for i, contract_type in enumerate(contract_type_categories):
        group = pos_cash[pos_cash['NAME_CONTRACT_STATUS'] == contract_type].groupby('SK_ID_PREV').agg(aggregations_for_categories)
        group.columns = ['_'.join(ele).upper() + '_' + contract_type.upper() for ele in group.columns]
        if i == 0:
            pos_cash_aggregated_contract = group
        else:
            pos_cash_aggregated_contract = pos_cash_aggregated_contract.merge(group, on='SK_ID_PREV', how='outer')

    # Merging all the aggregations
    pos_cash_aggregated = pos_cash_aggregated_overall.merge(pos_cash_aggregated_year, on='SK_ID_PREV', how='outer')
    pos_cash_aggregated = pos_cash_aggregated.merge(pos_cash_aggregated_contract, on='SK_ID_PREV', how='outer')

    # One-hot encoding the categorical feature NAME_CONTRACT_STATUS
    name_contract_dummies = pd.get_dummies(pos_cash['NAME_CONTRACT_STATUS'], prefix='CONTRACT')
    contract_names = name_contract_dummies.columns.tolist()
    # Concatenating one-hot encoded categories with main table
    pos_cash = pd.concat([pos_cash, name_contract_dummies], axis=1)
    # Aggregating these over SK_ID_PREV as well
    aggregated_cc_contract = pos_cash[['SK_ID_PREV'] + contract_names].groupby('SK_ID_PREV').mean()

    # Merging with the final aggregations
    pos_cash_aggregated = pos_cash_aggregated.merge(aggregated_cc_contract, on='SK_ID_PREV', how='outer')    
    return pos_cash_aggregated

def aggregations_sk_id_curr(pos_cash_aggregated):
    # Aggregating over SK_ID_CURR
    columns_to_aggregate = pos_cash_aggregated.columns[1:]
    # Defining the aggregations to perform
    aggregations_final = {}
    for col in columns_to_aggregate:
        if 'MEAN' in col:
            aggregates = ['mean', 'sum', 'max']
        else:
            aggregates = ['mean']
        aggregations_final[col] = aggregates
    pos_cash_aggregated_final = pos_cash_aggregated.groupby('SK_ID_CURR').agg(aggregations_final)
    pos_cash_aggregated_final.columns = ['_'.join(ele).upper() for ele in pos_cash_aggregated_final.columns]
    
    return pos_cash_aggregated_final


In [9]:
def pos_cash_balance_processing(pos_cash_balance):
    print("Start processing...")
    pos_cash_balance = cleaning_pos_cash(pos_cash_balance)
    pos_cash_balance = feature_engineering_pos_cash(pos_cash_balance)
    pos_cash_balance = aggregations_sk_id_prev(pos_cash_balance)
    pos_cash_balance = aggregations_sk_id_curr(pos_cash_balance)
    # Remove columns with more than 50% missing values
    pos_cash_balance = pos_cash_balance.loc[:, pos_cash_balance.isna().mean() < 0.5]
    print("Done!")
    return pos_cash_balance

In [10]:
POS_CASH_balance_final = pos_cash_balance_processing(POS_CASH_balance)



Start processing...
Done!


In [11]:
POS_CASH_balance_final.to_csv('POS_CASH_balance_final.csv', index=False)

In [12]:
POS_CASH_balance_final.head()

,MONTHS_BALANCE_MAX_MEAN,CNT_INSTALMENT_MEAN_MEAN,CNT_INSTALMENT_MEAN_SUM,CNT_INSTALMENT_MEAN_MAX,CNT_INSTALMENT_MAX_MEAN,CNT_INSTALMENT_MIN_MEAN,CNT_INSTALMENT_FUTURE_MEAN_MEAN,CNT_INSTALMENT_FUTURE_MEAN_SUM,CNT_INSTALMENT_FUTURE_MEAN_MAX,CNT_INSTALMENT_FUTURE_MAX_MEAN,CNT_INSTALMENT_FUTURE_MIN_MEAN,SK_DPD_MAX_MEAN,SK_DPD_SUM_MEAN,SK_DPD_DEF_MAX_MEAN,SK_DPD_DEF_SUM_MEAN,EXP_CNT_INSTALMENT_LAST_MEAN,EXP_CNT_INSTALMENT_FUTURE_LAST_MEAN,SK_DPD_RATIO_MEAN_MEAN,SK_DPD_RATIO_MEAN_SUM,SK_DPD_RATIO_MEAN_MAX,SK_DPD_RATIO_MAX_MEAN,CNT_INSTALMENT_MEAN_YEAR_0_SUM,CNT_INSTALMENT_FUTURE_MEAN_YEAR_0_SUM,SK_DPD_RATIO_MEAN_YEAR_0_SUM,CNT_INSTALMENT_MEAN_YEAR_1_SUM,...,CNT_INSTALMENT_MEAN_COMPLETED_SUM,CNT_INSTALMENT_MEAN_COMPLETED_MAX,CNT_INSTALMENT_MAX_COMPLETED_MEAN,CNT_INSTALMENT_MIN_COMPLETED_MEAN,CNT_INSTALMENT_FUTURE_MEAN_COMPLETED_MEAN,CNT_INSTALMENT_FUTURE_MEAN_COMPLETED_SUM,CNT_INSTALMENT_FUTURE_MEAN_COMPLETED_MAX,CNT_INSTALMENT_FUTURE_MAX_COMPLETED_MEAN,CNT_INSTALMENT_FUTURE_MIN_COMPLETED_MEAN,SK_DPD_MAX_COMPLETED_MEAN,SK_DPD_SUM_COMPLETED_MEAN,SK_DPD_DEF_MAX_COMPLETED_MEAN,SK_DPD_DEF_SUM_COMPLETED_MEAN,EXP_CNT_INSTALMENT_LAST_COMPLETED_MEAN,EXP_CNT_INSTALMENT_FUTURE_LAST_COMPLETED_MEAN,SK_DPD_RATIO_MEAN_COMPLETED_MEAN,SK_DPD_RATIO_MEAN_COMPLETED_SUM,SK_DPD_RATIO_MEAN_COMPLETED_MAX,SK_DPD_RATIO_MAX_COMPLETED_MEAN,CNT_INSTALMENT_MEAN_OTHERS_SUM,CNT_INSTALMENT_FUTURE_MEAN_OTHERS_SUM,SK_DPD_RATIO_MEAN_OTHERS_SUM,CONTRACT_ACTIVE_MEAN,CONTRACT_COMPLETED_MEAN,CONTRACT_OTHERS_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-4.250000,8.729167,34.916667,12.000000,10.000000,5.500000,7.416667,29.666667,12.000000,10.000000,3.00,0.0,0.0,0.0,0.0,9.765687,9.334254,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,10.0,5.0,3.333333,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.770833,0.229167,0.0
1,-27.500000,21.958333,43.916667,33.916667,23.000000,10.500000,16.708333,33.416667,28.416667,23.000000,0.00,7.5,14.5,0.0,0.0,22.999685,22.333350,120833.333333,241666.666667,241666.666667,750000.0,0.0,0.0,0.0,0.0,...,21.0,11.0,10.500000,10.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.912879,0.087121,0.0
2,-55.500000,7.500000,30.000000,12.000000,7.500000,7.500000,3.775000,15.100000,6.000000,7.500000,0.25,0.0,0.0,0.0,0.0,7.500000,6.852758,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,24.0,12.0,8.000000,8.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.880769,0.119231,0.0
3,-12.333333,9.522222,28.566667,10.833333,11.333333,3.666667,7.688889,23.066667,8.400000,11.333333,0.00,0.0,0.0,0.0,0.0,11.004050,10.431229,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,11.0,5.0,3.666667,3.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.766667,0.233333,0.0
4,-73.000000,6.000000,6.000000,6.000000,6.000000,6.000000,3.000000,3.000000,3.000000,6.000000,0.00,0.0,0.0,0.0,0.0,6.000000,5.344821,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,6.0,6.0,6.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.857143,0.142857,0.0


In [15]:
POS_CASH_balance_final.select_dtypes(include='object').columns


Index([], dtype='object')